Imports:

In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, Dot
from keras.layers.core import Dense, Reshape
import keras

import pandas as pd
import numpy as np
import ast
import tqdm
import pickle

RANDOM_SEED = 42
AUTOTUNE = tf.data.AUTOTUNE


2023-03-09 17:22:02.802390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 17:22:03.152734: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-09 17:22:03.152753: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-09 17:22:04.580515: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Import data

In [3]:
vocab = np.load("vocab.npy", allow_pickle=True)
X = np.load("training_data_X.pickle", allow_pickle=True)
Y = np.load("training_data_Y.pickle", allow_pickle=True)
vocab_size = len(vocab)

In [4]:
# X_targets = [
#     target_id for recipe in X for target_id in recipe[0]
# ]

# X_contexts = [
#     target_id for recipe in X for target_id in recipe[1]
# ]

count = 0
for recipe in X:
    for target_id in recipe[0]:
        count += 1

In [5]:
count

137483604

: 

# Constructing the Word2Vec model

In [5]:
# define vector size for embeddings
embedding_size = 100

First, we create the model to retrieve the target ingredient's embedding:

In [6]:
target_inputs = keras.Input(shape=(1,))

target_x = Embedding(
    # size of input vector - equal to vocab size
    input_dim=vocab_size,
    output_dim=embedding_size,
    # distribution to sample random values from for initial embeddings
    embeddings_initializer="glorot_uniform",
    input_length=1
)(target_inputs)
target_output = Reshape((embedding_size, ))(target_x)

target_model = keras.Model(inputs=target_inputs, outputs=target_output)

2023-03-08 20:47:01.126807: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-08 20:47:01.126837: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-08 20:47:01.126863: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jkunix): /proc/driver/nvidia/version does not exist
2023-03-08 20:47:01.127884: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Next we create the context ingredient's model:

In [7]:
context_inputs = keras.Input(shape=(1,))

context_x = Embedding(
    # size of input vector - equal to vocab size
    input_dim=vocab_size,
    output_dim=embedding_size,
    # distribution to sample random values from for initial embeddings
    embeddings_initializer="glorot_uniform",
    input_length=1
)(context_inputs)
context_output = Reshape((embedding_size, ))(context_x)

context_model = keras.Model(inputs=context_inputs, outputs=context_output)

We now combine the models together:

In [8]:
# take the dot product of the outputs of the target and context models
dot_layer = Dot(axes=1, normalize=False)([target_model.output, context_model.output])

# pass the dot product to a dense layer
combined_out = Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid")(dot_layer)

# compile to a model
combined_model = keras.Model(inputs=[target_model.input, context_model.input], outputs=combined_out)

In [9]:
combined_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 100)       735900      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 100)       735900      ['input_2[0][0]']                
                                                                                            

In [10]:
combined_model.compile(loss="categorical_crossentropy",optimizer="adam")

In [35]:
for i in tqdm.tqdm(range(len(X))):
    recipe_X = [
        np.array(X[i][0],dtype="int32"),
        np.array(X[i][1],dtype="int32")
    ]
    recipe_Y = np.array(Y[i],dtype="int32")
    combined_model.train_on_batch(
        recipe_X,
        recipe_Y
    )

100%|██████████| 511626/511626 [1:44:52<00:00, 81.31it/s]  


In [10]:
# X_tmp = [np.array([1,2,4,5],dtype="int32"), np.array([5,6,7,8],dtype="int32")]
# y_tmp = np.array([5,2,7,9])

# combined_model.fit(X_tmp,
#     y_tmp,
#     batch_size=1,
#     epochs=1
# )

4/4 [==============================] - 1s 18ms/step - loss: 0.0000e+00


In [11]:
ingredient_layer = combined_model.layers[3].get_weights()

In [37]:
np.save("weights.npy",ingredient_layer)

In [18]:
ingredient_layer[0][7000]

array([-0.00490222, -0.01940766, -0.01026326,  0.02755293,  0.01797247,
        0.00541672,  0.0281049 ,  0.01816593, -0.00531654, -0.02647945,
        0.00934397,  0.01540985, -0.01195652,  0.02025651,  0.01323678,
        0.00050395, -0.01363507, -0.0105191 , -0.00162502, -0.01784335,
       -0.01431565, -0.01396179,  0.0056959 ,  0.00654939, -0.02760641,
       -0.01432824,  0.02294516, -0.0192068 ,  0.0250996 , -0.01646177,
       -0.01655961, -0.00422387,  0.02272253, -0.00218097, -0.00265214,
        0.00980739, -0.01880799,  0.00310035,  0.00465292, -0.00456946,
       -0.00567238,  0.00936046, -0.00989753, -0.02441131, -0.00279154,
        0.02689251,  0.00120045,  0.00708014,  0.00061905, -0.00800823,
        0.00364976, -0.01847172, -0.00255303,  0.01344508,  0.02808063,
       -0.01439202, -0.01804162, -0.00081389,  0.01297373, -0.02634827,
       -0.00425972,  0.0213232 ,  0.02707435, -0.02066739,  0.0175138 ,
        0.00163177,  0.00017825,  0.01611352, -0.01804187,  0.00

# Archive

In [25]:
target_model = Sequential()
target_model.add(Embedding(
    # size of input vector - equal to vocab size
    input_dim=vocab_size,
    output_dim=embedding_size,
    # distribution to sapmle random values from for initial embeddings
    embeddings_initializer="glorot_uniform",
    input_length=1
))
target_model.add(Reshape((embedding_size,)))

context_model = Sequential()
context_model.add(Embedding(
    input_dim=vocab_size,
    output_dim=embedding_size,
    embeddings_initializer="glorot_uniform",
    input_length=1
))
context_model.add(Reshape((embedding_size,)))

model = Sequential()
# combine two embeddings with dot product

# context embedding input 
target_inp = keras.Input(shape=(embedding_size,))
context_inp = keras.Input(shape=(embedding_size,))

model.add(Dot(axes=(embedding_size,embedding_size),normalize=False)([target_model, context_model]))
model.add(Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid"))
model.compile(loss="mean_squared_error",optimizer="rmsprop")
            

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# # length equal to number of recipes
# skipgrams = [
#     # each row is a recipe
#     [
#         # first index contains word pairs of (target, context) word
#         [
#             (target_word_id, context_word_id), ...,    
#         ], 
#         # second index contains labels for the context word
#         [
#             0, 1, 0, 0, 0, ...
#         ]
#     ]
# ]

At training:

In [6]:
# get a single recipe

# get a list of all target words

# get a list of all context words

# get a list of labels for context words

# let [list_of_targets, list_of_contexts] be features, X

# let labels be Y

# fit on X and Y

# spread the pairs
list(zip(*[(1,2),(3,4),(5,6)]))

[(1, 3, 5), (2, 4, 6)]